In [1]:
import numpy as np
import pandas as pd
import torch


In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [53]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [7]:
len(y_cookie)

552

In [56]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [8]:
y_sentence = torch.load('sentence_target_dict.pt')

In [9]:
len(y_sentence)

240

Load Samples

In [10]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [11]:
len(X_cookie_filenames)

552

In [12]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [13]:
len(X_sent_filenames)

240

In [14]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [15]:
num_samples

792

In [16]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

Load Vocab Embeddings

In [17]:
glove_path = './PretrainedWordEmb/'

In [18]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [19]:
pos_dict = torch.load('pos_dict.pt')

In [52]:
def get_minibatch(batchsize, ix=None):
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
            print(file_ix, file)
            print('Words',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
            
    return embeddings, torch.tensor(targets)

In [53]:
e,t = get_minibatch(10)

216 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/298-1.cha
Words 100
312 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/089-0.cha
Words 172
108 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/130-3.cha
Words 63
153 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/175-1.cha
Words 187
481 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/488-1.cha
Words 34
201 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/274-1.cha
Words 80
50 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/068-2.cha
Words 104
432 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/325-0.cha
Words 87
421 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/282-2.cha
Words 141
223 /Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/318-2.cha
Words 76


In [54]:
[len(emb) for emb in e]

[100, 172, 63, 187, 34, 80, 104, 87, 141, 76]

In [55]:
t

tensor([29, 24, 29, 30, 17, 26, 30, 20, 23, 30])

In [69]:
minibatch_ix = random.sample(range(num_samples),5)